In [1]:
import numpy as np
import pickle
from scipy import signal
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import copy
import matplotlib.pyplot as plt
from numpy import cos, sin, pi, absolute, arange
from scipy.signal import kaiserord, lfilter, firwin, freqz
from pylab import figure, clf, plot, xlabel, ylabel, xlim, ylim, title, grid, axes, show
import numpy as np
from scipy.fftpack import fft, ifft
import scipy.signal as signal
import numpy as np
import pickle
from scipy import signal
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import copy
import matplotlib.pyplot as plt
%matplotlib inline
import pytz # new import
import datetime
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import preprocessing
from sklearn import utils
import copy
from copy import deepcopy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from function import *
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier

In [2]:
def log_(alpha,dtw,label,names):
	index = 14
	label = label[:index] + label[index+1 :]
	names = names[:index] + names[index+1 :]
	X = dtw
	y = np.asanyarray(label)
	loo = LeaveOneOut()
	loo.get_n_splits(X)
	count = 0
	correct_sub = []
	fp_sub = []
	fn_sub = []
	for train_index, test_index in loo.split(X):
		#print (train_index,test_index)
		X_train, X_test = X[train_index], X[test_index]
		y_train, y_test = y[train_index], y[test_index]
		scaler = StandardScaler() 
		scaler.fit(X_train) 
		X_train = scaler.transform(X_train)  
		X_test = scaler.transform(X_test) 
		#clf = SVC(C = alpha)
		clf = linear_model.LogisticRegression(C=alpha)
		clf.fit(X_train, y_train)  
		Y_predict = clf.predict(X_test)
		if Y_predict == y_test:
			#correct_sub.append(names[test_index])
			count += 1
		else:
			if Y_predict == 1:
				#print (test_index)
				fp_sub.append(names[int(test_index)])
			elif Y_predict == 0:
				fn_sub.append(names[int(test_index)])
		#print (Y_predict)
	return count/(len(X)),fp_sub,fn_sub


def pick_best_fn_fp_(distance_new,label,names):	
	alpha = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
	record = []
	fp_sub_list = []
	fn_sub_list = []
	for x in alpha:
	    #print (10**(x))
	    rate,fp_sub,fn_sub = log_(10**x,distance_new,label,names)
	    record.append(rate)
	    fp_sub_list.append(fp_sub)
	    fn_sub_list.append(fn_sub)
	max_record = max(record)
	index_max = record.index(max_record)
	#print (index_max)
	alpha_max = 10**alpha[index_max]
	fp_sub_max = fp_sub_list[index_max]
	fn_sub_max = fn_sub_list[index_max]
	return max_record,alpha_max,fp_sub_max,fn_sub_max



## Config

In [15]:
path = '0820_data/minute_data_0820'
start_train,end_train = 15, 17
start_test,end_test = 17, 18
start_hour,end_hour = '09','08'
thre = 100
names = ['HRV15-002', 'HRV15-003', 'HRV15-004', 'HRV15-005', 'HRV15-006', 'HRV15-007', 'HRV15-008', 'HRV15-009', 'HRV15-011', 'HRV15-012', 'HRV15-013',  'HRV15-017', 'HRV15-018', 'HRV15-019', 'HRV15-020', 'HRV15-021', 'HRV15-022', 'HRV15-023', 'HRV15-024']
label = pickle.load(open("labels.pkl", "rb" )).tolist()

In [16]:
start_hour,end_hour = '09','08'
n = 1
rand_state = 1234
#dtw = result_1509[-1]
result_1509 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [25]:
# Accuracy
print ('accuracy:',result_1509[0])
print ('FP:',result_1509[2])
print ('FN:',result_1509[3])
##

accuracy: 0.6666666666666666
FP: ['HRV15-006', 'HRV15-007', 'HRV15-008', 'HRV15-013', 'HRV15-021', 'HRV15-022']
FN: []


In [20]:
result_1509[4].shape


(19, 3)

In [6]:
start_train

15

In [17]:
# FP
result_1509[2]

['HRV15-006', 'HRV15-008', 'HRV15-013', 'HRV15-021', 'HRV15-022', 'HRV15-024']

In [18]:
# FN
result_1509[3]

['HRV15-005']

In [ ]:
def svm_(gamma,dtw,label,names):
	index = 14
	label = label[:index] + label[index+1 :]
	names = names[:index] + names[index+1 :]
	X = dtw
	y = np.asanyarray(label)
	loo = LeaveOneOut()
	loo.get_n_splits(X)
	count = 0
	correct_sub = []
	fp_sub = []
	fn_sub = []
	for train_index, test_index in loo.split(X):
		#print (train_index,test_index)
		X_train, X_test = X[train_index], X[test_index]
		y_train, y_test = y[train_index], y[test_index]
		scaler = StandardScaler() 
		scaler.fit(X_train) 
		X_train = scaler.transform(X_train)  
		X_test = scaler.transform(X_test) 
		#clf = svm.SVC(kernel='rbf',gamma = gamma)
		#clf = MLPClassifier(solver='lbfgs', alpha=gamma, hidden_layer_sizes=(10, ), random_state=1)
		#clf = svm.SVC(kernel='poly', degree=5, C=gamma)
		#clf = linear_model.LogisticRegression(C=alpha)
		clf.fit(X_train, y_train)  
		Y_predict = clf.predict(X_test)
		if Y_predict == y_test:
			#correct_sub.append(names[test_index])
			count += 1
		else:
			if Y_predict == 1:
				#print (test_index)
				fp_sub.append(names[int(test_index)])
			elif Y_predict == 0:
				fn_sub.append(names[int(test_index)])
		#print (Y_predict)
	return count/(len(X)),fp_sub,fn_sub

In [ ]:
def pick_best_fn_fp(distance_new,label,names):	
	gamma = np.arange(0.00001,1000,10)
	record = []
	fp_sub_list = []
	fn_sub_list = []
	for x in gamma:
	    #print (10**(x))
	    rate,fp_sub,fn_sub = svm_(x,distance_new,label,names)
	    record.append(rate)
	    fp_sub_list.append(fp_sub)
	    fn_sub_list.append(fn_sub)
	max_record = max(record)
	index_max = record.index(max_record)
	alpha_max = gamma[index_max]
	fp_sub_max = fp_sub_list[index_max]
	fn_sub_max = fn_sub_list[index_max]
	return max_record,alpha_max,fp_sub_max,fn_sub_max

In [18]:
start_train,end_train = 15, 17
start_test,end_test = 17, 18
start_hour,end_hour = '10','09'
n = 2
result_1510 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [20]:
result_1510[0]

0.5555555555555556

In [19]:
start_hour,end_hour = '11','10'
n = 3

result_1511 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [21]:
result_1511[0]

0.5555555555555556

In [ ]:
distance_new = result_1511[-1]
pick_best_fn_fp_(distance_new,label,names)

In [12]:
start_hour,end_hour = '12','11'
n = 4
result_1512 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [13]:
start_hour,end_hour = '13','12'
n = 5
result_1513 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
start_hour,end_hour = '14','13'
n = 6
result_1514 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
start_hour,end_hour = '15','14'
n = 7
result_1515 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
result_1515[0]

In [ ]:
start_hour,end_hour = '16','15'
n = 8
result_1516 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
result_1516[2]

In [ ]:
start_hour,end_hour = '17','16'
n = 9
result_1517 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
start_hour,end_hour = '18','17'
n = 10
result_1518 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
start_hour,end_hour = '19','18'
n = 11
result_1519 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
distance_new = result_1519[-1]
pick_best_fn_fp_(distance_new,label,names)

In [ ]:
result_1510[2]

In [ ]:
start_hour,end_hour = '20','19'
n = 12
result_1520 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
start_hour,end_hour = '08','07'
n = 13
result_1508 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n,rand_state)

In [ ]:
result_1520

In [ ]:
print ('(fp,true_0,pred_1)',result_1520[2])
print('(fn,true_1,pred_0)',result_1520[3])

In [ ]:
# start_train,end_train = 14, 16
# start_test,end_test = 16, 17

# start_hour,end_hour = '19','20'
# n = 14
# result_1420 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n)

In [ ]:
start_hour,end_hour = '06','06'
n = 14
result_1507 = result_time_shift(path,start_train,end_train,start_test,end_test,start_hour,end_hour,thre,names,label,n)

In [ ]:
start_hour_list = ['07','08','09','10','11','12','13','14','15','16','17']
end_hour_list = ['06','07','08','09','10','11','12','13','14','15','16']

In [ ]:
result_total = []
for i in range(len(start_hour_list)):
    r =  result_time_shift(path,start_train,end_train,start_test,end_test,start_hour_list[i],end_hour_list[i],thre,names,label,i)
    result_total.append(r)
    print (i)

In [ ]:

alpha = [-2,-1,0,1,2,3,4,5]
record = []
fp_sub_list = []
fn_sub_list = []
for x in alpha:
    #print (10**(x))
    rate,fp_sub,fn_sub = svm_(10**x,distance_new,label,names)
    record.append(rate)
    fp_sub_list.append(fp_sub)
    fn_sub_list.append(fn_sub)
max_record = max(record)
index_max = record.index(max_record)
alpha_max = 10**alpha[index_max]
fp_sub_max = fp_sub_list[index_max]
fn_sub_max = fn_sub_list[index_max]



In [ ]:
fn_sub_max

In [ ]:
max_record

In [ ]:
test_index

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.ioff()

plt.plot([1,2,3])
plt.savefig('/tmp/test.png')